In [1]:
%load_ext lab_black

In [2]:
import io
import requests
import re
import zipfile
import pathlib
import time
import datetime

import numpy as np

from IPython.display import clear_output
from itertools import chain
from psycopg2 import connect
from sqlalchemy import create_engine

In [3]:
def get_master_file_list(url):
    r = requests.get(url)
    if not r.ok:
        raise BaseException(f"Invalid HTML status {r.status_code}")
    raw_list = r.text.split("\n")
    gkg_list = []
    event_list = []
    for raw in raw_list:
        try:
            file_location = re.search(r"https?://[^\s]+", raw).group()
        except AttributeError:
            pass

        if ".gkg" in file_location:
            gkg_list.append(file_location)

        if ".export" in file_location:
            event_list.append(file_location)

    return gkg_list, event_list


def fetch_and_parse_zip(url):
    r = requests.get(url)
    if not r.ok:
        csv = open("/tmp/gdelt_csv_empty_file.csv", "w+")
        return csv
    file = r.content
    zip_object = zipfile.ZipFile(io.BytesIO(file), "r")
    csv = zip_object.open(zip_object.namelist()[0])

    return csv


def get_relevant_events_lines(content, words, aggregate_file):
    for line in content.readlines():
        try:
            line = line.decode("utf-8")
            source_url = line.split("\t")[-1].strip("\n")
            terms_in_url = re.split(r"[-._/]+", source_url)
            if any([word in terms_in_url for word in words]):
                aggregate_file.write(line)
        except UnicodeDecodeError:
            pass

    return


def get_gdelt_events_data(words, progress_file, file_list):
    processed_files = [pf.strip() for pf in progress_file.readlines()]
    N = len(file_list)
    p = pathlib.Path(f"../data")
    p.mkdir(parents=True, exist_ok=True)
    q = p / "gdelt_events.csv"
    with q.open("a+") as agg:
        start_time = time.time()
        m = 0
        for n, file_url in enumerate(file_list):
            if file_url in processed_files:
                m += 1
                continue
            csv = fetch_and_parse_zip(file_url)
            get_relevant_events_lines(csv, words, agg)

            clear_output(wait=True)

            elapsed_time = time.time() - start_time

            elapsed_time_tuple = str(datetime.timedelta(seconds=elapsed_time)).split(
                ":"
            )

            elapsed_time_string = f"{elapsed_time_tuple[0]}:{elapsed_time_tuple[1]}:{round(float(elapsed_time_tuple[2])):02}"

            estimated_time_remaining = (
                elapsed_time * (N - m) / (n - m + 1)
            ) - elapsed_time

            estimated_remaining_time_tuple = str(
                datetime.timedelta(seconds=estimated_time_remaining)
            ).split(":")

            estimated_remaining_time_string = f"{estimated_remaining_time_tuple[0]}:{estimated_remaining_time_tuple[1]}:{round(float(estimated_remaining_time_tuple[2])):02}"

            print(f"{n + 1}/{N} files parsed")
            print(f"Elapsed time: {elapsed_time_string}")
            print(f"Estimated remaining time: {estimated_remaining_time_string}")

            progress_file.write(file_url + "\n")

    return

In [4]:
gkg_list, events_list = get_master_file_list(
    "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
)

In [5]:
OIL_WORDS = [
    "oil",
    "gas",
    "gasoline",
    "petrol",
    "fuel",
    "petroleum",
    "diesel",
]

In [6]:
try:
    with open("../data/events_progress.txt", "r+") as progress_file:
        get_gdelt_events_data(OIL_WORDS, progress_file, events_list)
except FileNotFoundError:
    with open("../data/events_progress.txt", "w+") as progress_file:
        get_gdelt_events_data(OIL_WORDS, progress_file, events_list)

272839/272839 files parsed
Elapsed time: 0:10:20
Estimated remaining time: 0:00:00


# SQL Hosting

In [7]:
%%capture
! sudo -S -i -u postgres dropdb -f gdelt < ../etc/user.password
! sudo -S -i -u postgres createdb gdelt < ../etc/user.password

In [8]:
event_columns = """id serial PRIMARY KEY,
                   GlobalEventID integer, 
                   Day integer,
                   MonthYear integer,
                   Year integer,
                   FractionDate numeric,
                   Actor1Code text,
                   Actor1Name text,
                   Actor1CountryCode text,
                   Actor1KnownGroupCode text,
                   Actor1EthnicCode text,
                   Actor1Religion1Code text,
                   Actor1Religion2Code text,
                   Actor1Type1Code text,
                   Actor1Type2Code text,
                   Actor1Type3Code text,
                   Actor2Code text,
                   Actor2Name text,
                   Actor2CountryCode text,
                   Actor2KnownGroupCode text,
                   Actor2EthnicCode text,
                   Actor2Religion1Code text,
                   Actor2Religion2Code text,
                   Actor2Type1Code text,
                   Actor2Type2Code text,
                   Actor2Type3Code text,
                   IsRootEvent integer,
                   EventCode text,
                   EventBaseCode text,
                   EventRootCode text,
                   QuadClass integer,
                   GoldsteinScale text,
                   NumMentions integer,
                   NumSources integer,
                   NumArticles integer,
                   AvgTone numeric,
                   Actor1Geo_Type integer,
                   Actor1Geo_Fullname text,
                   Actor1Geo_CountryCode text,
                   Actor1Geo_ADM1Code text,
                   Actor1Geo_ADM2Code text,
                   Actor1Geo_Lat text,
                   Actor1Geo_Long text,
                   Actor1Geo_FeatureID text,
                   Actor2Geo_Type integer,
                   Actor2Geo_Fullname text,
                   Actor2Geo_CountryCode text,
                   Actor2Geo_ADM1Code text,
                   Actor2Geo_ADM2Code text,
                   Actor2Geo_Lat text,
                   Actor2Geo_Long text,
                   Actor2Geo_FeatureID text,
                   ActionGeo_Type integer,
                   ActionGeo_Fullname text,
                   ActionGeo_CountryCode text,
                   ActionGeo_ADM1Code text,
                   ActionGeo_ADM2Code text,
                   ActionGeo_Lat text,
                   ActionGeo_Long text,
                   ActionGeo_FeatureID text,
                   DATEADDED bigint,
                   SOURCEURL text"""

# NOTE: strictly speaking, many columns (such as GoldsteinScale)
# should be of type numeric. However, these columns contain
# empty strings in the .csv files. This will cause issues
# in the COPY FROM postgreSQL command. Therefore, we treat
# these columns as text initially. If reason is found to use
# these columns in the analysis, the columns can be converted later
# within the database or via python after the data is fetched

event_columns_no_types_no_id = (
    event_columns.replace(" text", "")
    .replace(" numeric", "")
    .replace(" bigint", "")
    .replace(" integer", "")
    .replace("id serial PRIMARY KEY,", "")
)

In [11]:
with open("../data/gdelt_events.csv", "r") as events_file, open(
    "../etc/postgres.password"
) as psql_pass_file:
    postgres_password = psql_pass_file.read()
    print(postgres_password)
    conn = connect(
        f"host='localhost' dbname='gdelt' user='postgres' password='{postgres_password}'"
    )
    cursor = conn.cursor()
    create_events_table_cmd = f"CREATE TABLE events({event_columns})"
    copy_events_cmd = f"COPY events({event_columns_no_types_no_id}) FROM STDIN WITH (FORMAT TEXT, HEADER FALSE)"
    cursor.execute(create_events_table_cmd)
    cursor.copy_expert(copy_events_cmd, events_file)
    conn.commit()

postgres
